# Decision trees

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv('titanic-homework.csv')
df.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22,1,0,0


In [3]:
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived']]

# Version for 4.0

### Categorical features

In [4]:
bins = [0, 20, 40, 100]
labels = ['young', 'medium', 'old']
df['Age'] = pd.cut(df['Age'], bins=bins, labels=labels)

for column in df.columns:
    df[column] = df[column].astype('category')

### Entropy
$H(S) = - \sum\limits_{y \in S}p(y)log_2p(y)$ <br> <br>
$U=|S|$

In [5]:
def entropy(X, S):
    U, H = len(X), 0
    for Ui in X[S].value_counts():
        p = Ui / U
        if p > 0:
            H += (-p * np.log2(p))
    return H

### Conditional entropy

$H(S|A) = - \sum\limits_{x \in A, y \in S} p(x,y) log_2 \frac{p(x,y)}{p(x)}$

In [6]:
def conditional_entropy(X, S, A):
    U, H = len(X), 0
    for Ai in X[A].unique():
        px = len(X[X[A] == Ai]) / U
        if px <= 0: continue

        for Si in X[S].unique():
            pxy = len(X[(X[A] == Ai) & (X[S] == Si)]) / U
            if pxy > 0:
                H += (-pxy * np.log2(pxy/px))
    return H

### Information gain
$IG(S,A) = H(S) - H(S|A)$

In [7]:
def information_gain(X, S, A):
    IG = entropy(X, S) - conditional_entropy(X, S, A)
    return IG

### Intrinsic information
$II(S,A)=- \sum\limits_{x \in A, y \in S} p(x,y) log_2 p(x,y)$

In [8]:
def intrinsic_info(X, S, A):
    U, info = len(X), 0
    for Ui in X[A].value_counts():
        pxy = Ui / U
        if pxy > 0:
            info += (-pxy * np.log2(pxy))
    return info

### Gain ratio
$IGR(S,A) = \frac{IG(S,A)}{II(S,A)}$

In [9]:
def gain_ratio(X, S, A):
    IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
    return IGR

### ID3

In [10]:
target = 'Survived'
attributes = df.drop(columns=(target)).columns

def split_and_go_deep(X, attr, depth=0):
    
    print(f'DEPTH: {depth}')
    
    if len(attr) == 0:
        print(f'\n\n======LEAF=======!\n\n')
        return
    
    gain_ratios = np.array([gain_ratio(X, target, a) for a in attr])
    i = np.argmax(gain_ratios)
    best_attr, best_ratio = attr[i], gain_ratios[i]
    print(f'Attr split: {best_attr}\nGain ratio: {best_ratio}\n\n')    
    
    attr = attr.drop(best_attr)
    
    for var in X[best_attr].unique():
        X_copy = X[X[best_attr] == var]
        print(X_copy.head(3), '\n\n')
        split_and_go_deep(X_copy, attr, depth + 1)

split_and_go_deep(df, attributes)

DEPTH: 0
Attr split: Sex
Gain ratio: 0.40323636523376327


  Pclass   Sex     Age SibSp Parch Survived
0      3  male  medium     1     0        0
4      3  male  medium     0     0        0
5      3  male  medium     0     0        0 


DEPTH: 1
Attr split: Pclass
Gain ratio: 0.07729306659385596


  Pclass   Sex     Age SibSp Parch Survived
0      3  male  medium     1     0        0
4      3  male  medium     0     0        0
5      3  male  medium     0     0        0 


DEPTH: 2
Attr split: Parch
Gain ratio: 0.07483597867418908


  Pclass   Sex     Age SibSp Parch Survived
0      3  male  medium     1     0        0
4      3  male  medium     0     0        0
5      3  male  medium     0     0        0 


DEPTH: 3
Attr split: Age
Gain ratio: 0.0


  Pclass   Sex     Age SibSp Parch Survived
0      3  male  medium     1     0        0
4      3  male  medium     0     0        0
5      3  male  medium     0     0        0 


DEPTH: 4
Attr split: SibSp
Gain ratio: 0.0


   Pclass   Se

/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid val

   Pclass   Sex    Age SibSp Parch Survived
86      3  male  young     1     3        0 


DEPTH: 5


======LEAF=======!


   Pclass   Sex     Age SibSp Parch Survived
6       1  male     old     0     0        0
23      1  male  medium     0     0        1
27      1  male   young     3     2        0 


DEPTH: 2
Attr split: Age
Gain ratio: 0.34308005154023885


   Pclass   Sex  Age SibSp Parch Survived
6       1  male  old     0     0        0
35      1  male  old     1     0        0
54      1  male  old     0     1        0 


DEPTH: 3
Attr split: SibSp
Gain ratio: 0.0


   Pclass   Sex  Age SibSp Parch Survived
6       1  male  old     0     0        0
54      1  male  old     0     1        0
64      1  male  old     0     0        0 


DEPTH: 4
Attr split: Parch
Gain ratio: 0.0


   Pclass   Sex  Age SibSp Parch Survived
6       1  male  old     0     0        0
64      1  male  old     0     0        0
96      1  male  old     0     0        0 


DEPTH: 5


======LEAF=======!




/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid val

Attr split: Parch
Gain ratio: nan


   Pclass   Sex  Age SibSp Parch Survived
33      2  male  old     0     0        0 


DEPTH: 5


======LEAF=======!


  Pclass     Sex     Age SibSp Parch Survived
1      1  female  medium     1     0        1
2      3  female  medium     0     0        1
3      1  female  medium     1     0        1 


DEPTH: 1
Attr split: SibSp
Gain ratio: 0.16649361093302306


  Pclass     Sex     Age SibSp Parch Survived
1      1  female  medium     1     0        1
3      1  female  medium     1     0        1
9      2  female   young     1     0        1 


DEPTH: 2
Attr split: Pclass
Gain ratio: 0.29977231627095335


   Pclass     Sex     Age SibSp Parch Survived
1       1  female  medium     1     0        1
3       1  female  medium     1     0        1
31      1  female     old     1     0        1 


DEPTH: 3
Attr split: Parch
Gain ratio: nan


   Pclass     Sex     Age SibSp Parch Survived
1       1  female  medium     1     0        1
3       1  female 

/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)


Attr split: Age
Gain ratio: nan


   Pclass     Sex     Age SibSp Parch Survived
25      3  female  medium     1     5        1 


DEPTH: 5


======LEAF=======!


   Pclass     Sex     Age SibSp Parch Survived
2       3  female  medium     0     0        1
8       3  female  medium     0     2        1
11      1  female     old     0     0        1 


DEPTH: 2
Attr split: Pclass
Gain ratio: 0.0


   Pclass     Sex     Age SibSp Parch Survived
2       3  female  medium     0     0        1
8       3  female  medium     0     2        1
14      3  female   young     0     0        1 


DEPTH: 3
Attr split: Age
Gain ratio: 0.0


   Pclass     Sex     Age SibSp Parch Survived
2       3  female  medium     0     0        1
8       3  female  medium     0     2        1
79      3  female  medium     0     0        1 


DEPTH: 4
Attr split: Parch
Gain ratio: 0.0


   Pclass     Sex     Age SibSp Parch Survived
2       3  female  medium     0     0        1
79      3  female  medium     0     

/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid val

   Pclass     Sex    Age SibSp Parch Survived
24      3  female  young     3     1        0 


DEPTH: 4
Attr split: Parch
Gain ratio: nan


   Pclass     Sex    Age SibSp Parch Survived
24      3  female  young     3     1        0 


DEPTH: 5


======LEAF=======!


   Pclass     Sex     Age SibSp Parch Survived
85      3  female  medium     3     0        1
88      1  female  medium     3     2        1 


DEPTH: 3
Attr split: Pclass
Gain ratio: 0.0


   Pclass     Sex     Age SibSp Parch Survived
85      3  female  medium     3     0        1 


DEPTH: 4
Attr split: Parch
Gain ratio: nan


   Pclass     Sex     Age SibSp Parch Survived
85      3  female  medium     3     0        1 


DEPTH: 5


======LEAF=======!


   Pclass     Sex     Age SibSp Parch Survived
88      1  female  medium     3     2        1 


DEPTH: 4
Attr split: Parch
Gain ratio: nan


   Pclass     Sex     Age SibSp Parch Survived
88      1  female  medium     3     2        1 


DEPTH: 5


======LEAF=======!


 

/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid value encountered in scalar divide
  IGR = information_gain(X, S, A) / intrinsic_info(X, S, A)
/tmp/ipykernel_9562/2448395234.py:2: RuntimeWarning: invalid val

In [11]:
class Node():
    def Node(self, split_attr):
        self.split_attr = split_attr 
        
class DecisionTree():
    pass

### Testing

In [12]:
target = 'Survived'
attr = 'Sex'

print(entropy(df, attr))
print(conditional_entropy(df, target, attr)) # H(S|A) <= H(S)
print(information_gain(df, target, attr))
print(intrinsic_info(df, target, attr))
print(gain_ratio(df, target, attr))

0.9709505944546686
0.5794280059252063
0.3915225885294623
0.9709505944546686
0.40323636523376327


# Version for 5.0